# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
import csv
URL= r'C:\Users\carol\Ironhack\2.3-lab_df_calculation_and_transformation\your-code\Orders.csv'

In [3]:
orders_original = pd.read_csv(URL)


In [4]:
orders= orders_original.copy()
orders

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


In [5]:
orders.columns

Index(['Unnamed: 0', 'InvoiceNo', 'StockCode', 'year', 'month', 'day', 'hour',
       'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID',
       'Country', 'amount_spent'],
      dtype='object')

In [6]:
orders.dtypes

Unnamed: 0        int64
InvoiceNo         int64
StockCode        object
year              int64
month             int64
day               int64
hour              int64
Description      object
Quantity          int64
InvoiceDate      object
UnitPrice       float64
CustomerID        int64
Country          object
amount_spent    float64
dtype: object

In [7]:
orders.describe()

,Unnamed: 0,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,CustomerID,amount_spent
count,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000
mean,278465.221859,560617.126645,2010.934259,7.612537,3.614555,12.728247,13.021823,3.116174,15294.315171,22.394749
std,152771.368303,13106.167695,0.247829,3.416527,1.928274,2.273535,180.420210,22.096788,1713.169877,309.055588
min,0.000000,536365.000000,2010.000000,1.000000,1.000000,6.000000,1.000000,0.000000,12346.000000,0.000000
25%,148333.750000,549234.000000,2011.000000,5.000000,2.000000,11.000000,2.000000,1.250000,13969.000000,4.680000
50%,284907.500000,561893.000000,2011.000000,8.000000,3.000000,13.000000,6.000000,1.950000,15159.000000,11.800000
75%,410079.250000,572090.000000,2011.000000,11.000000,5.000000,14.000000,12.000000,3.750000,16795.000000,19.800000
max,541908.000000,581587.000000,2011.000000,12.000000,7.000000,20.000000,80995.000000,8142.750000,18287.000000,168469.600000


In [8]:
orders.info('memory==usage')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397924 entries, 0 to 397923
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    397924 non-null  int64  
 1   InvoiceNo     397924 non-null  int64  
 2   StockCode     397924 non-null  object 
 3   year          397924 non-null  int64  
 4   month         397924 non-null  int64  
 5   day           397924 non-null  int64  
 6   hour          397924 non-null  int64  
 7   Description   397924 non-null  object 
 8   Quantity      397924 non-null  int64  
 9   InvoiceDate   397924 non-null  object 
 10  UnitPrice     397924 non-null  float64
 11  CustomerID    397924 non-null  int64  
 12  Country       397924 non-null  object 
 13  amount_spent  397924 non-null  float64
dtypes: float64(2), int64(8), object(4)
memory usage: 42.5+ MB


In [9]:
orders.shape

(397924, 14)

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [10]:
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [11]:
orders.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Unnamed: 0,397924.0,NaN,NaN,NaN,278465.221859,152771.368303,0.0,148333.75,284907.5,410079.25,541908.0
InvoiceNo,397924.0,NaN,NaN,NaN,560617.126645,13106.167695,536365.0,549234.0,561893.0,572090.0,581587.0
StockCode,397924,3665,85123A,2035,NaN,NaN,NaN,NaN,NaN,NaN,NaN
year,397924.0,NaN,NaN,NaN,2010.934259,0.247829,2010.0,2011.0,2011.0,2011.0,2011.0
month,397924.0,NaN,NaN,NaN,7.612537,3.416527,1.0,5.0,8.0,11.0,12.0
day,397924.0,NaN,NaN,NaN,3.614555,1.928274,1.0,2.0,3.0,5.0,7.0
hour,397924.0,NaN,NaN,NaN,12.728247,2.273535,6.0,11.0,13.0,14.0,20.0
Description,397924,3877,white hanging heart t-light holder,2028,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Quantity,397924.0,NaN,NaN,NaN,13.021823,180.42021,1.0,2.0,6.0,12.0,80995.0
InvoiceDate,397924,17286,2011-11-14 15:27:00,542,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#Resolver subproblema 1: Cómo agregar el 'amount_spent' para clientes únicos.
gastos_agregados_clientes=orders.groupby('CustomerID')['amount_spent'].sum()
gastos_agregados_clientes

CustomerID
12346    77183.60
12347     4310.00
12348     1797.24
12349     1757.55
12350      334.40
           ...   
18280      180.60
18281       80.82
18282      178.05
18283     2094.88
18287     1837.28
Name: amount_spent, Length: 4339, dtype: float64

In [13]:
#Resolver subproblema 2: Cómo seleccionar clientes cuyos gastos agregados estén en un rango de cuantiles.
porcentaje_75=gastos_agregados_clientes.quantile()
porcentaje_95=gastos_agregados_clientes.quantile()

In [14]:
vip_customers= gastos_agregados_clientes[gastos_agregados_clientes > porcentaje_95]
preferred_customers= gastos_agregados_clientes[(porcentaje_75 >= gastos_agregados_clientes) & (gastos_agregados_clientes< porcentaje_95)]
regular_customers= [gastos_agregados_clientes < porcentaje_75]

In [15]:
orders.head().T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
InvoiceNo,536365,536365,536365,536365,536365
StockCode,85123A,71053,84406B,84029G,84029E
year,2010,2010,2010,2010,2010
month,12,12,12,12,12
day,3,3,3,3,3
hour,8,8,8,8,8
Description,white hanging heart t-light holder,white metal lantern,cream cupid hearts coat hanger,knitted union flag hot water bottle,red woolly hottie white heart.
Quantity,6,6,8,6,6
InvoiceDate,2010-12-01 08:26:00,2010-12-01 08:26:00,2010-12-01 08:26:00,2010-12-01 08:26:00,2010-12-01 08:26:00


In [16]:
#Resolver problema 3: Cómo etiqueta a los clientes seleccionados como 'VIP' o 'Preferred'.
orders['CustomerType'] = 'Regular'  # Etiqueto a todos los clientes como "Regular" por defecto
orders.loc[orders['CustomerID'].isin(vip_customers.index), 'CustomerType'] = 'VIP'  # Etiqueto a los clientes VIP
orders.loc[orders['CustomerID'].isin(preferred_customers.index), 'CustomerType'] = 'Preferred'  # Etiqueto a los clientes Preferred


In [21]:
orders['CustomerType'].value_counts() #aquí especifico cuantos hay de cada categoría

VIP          339120
Preferred     58765
Regular          39
Name: CustomerType, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

Para responder esta pregunta voy a seguir los siguientes pasos:
1.Agruparé los datos por país y tipo de cliente(VIP, Preferred, Regular)
2.Contaré la cantidad de clientes VIP en cada país.
Encontraré el país con la mayoría de clientes vip.

In [22]:
orders.columns

Index(['Unnamed: 0', 'InvoiceNo', 'StockCode', 'year', 'month', 'day', 'hour',
       'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID',
       'Country', 'amount_spent', 'CustomerType'],
      dtype='object')

In [27]:
# Agrupo los datos por país y tipo de cliente para contar la cantidad de clientes VIP en cada país
country_vip_counts=orders[orders['CustomerType']=='VIP'].groupby('Country')['CustomerID'].nunique()
country_vip_counts

Country
Australia                  7
Austria                    8
Belgium                   19
Brazil                     1
Canada                     1
Channel Islands            8
Cyprus                     4
Czech Republic             1
Denmark                    6
EIRE                       3
European Community         1
Finland                    6
France                    50
Germany                   61
Greece                     3
Iceland                    1
Israel                     2
Italy                      8
Japan                      5
Lebanon                    1
Lithuania                  1
Malta                      2
Netherlands                4
Norway                     7
Poland                     3
Portugal                  10
RSA                        1
Singapore                  1
Spain                     19
Sweden                     4
Switzerland               17
USA                        2
United Arab Emirates       2
United Kingdom          1907
Unspec

In [25]:
# Encuentro el país con la mayoría de clientes VIP
country_with_most_vip = country_vip_counts.idxmax() #me devolverá la etiqueta del país con la mayoría de clientes VIP.
number_of_vip_customers = country_vip_counts.max()

In [26]:
print(f'El país con la mayoría de clientes VIP es {country_with_most_vip} con {number_of_vip_customers}')

El país con la mayoría de clientes VIP es United Kingdom con 1907


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [28]:
country_vip_preferred_counts = orders[orders['CustomerType'].isin(['VIP', 'Preferred'])].groupby('Country')['CustomerID'].nunique()


In [29]:
country_with_most_vip_preferred = country_vip_preferred_counts.idxmax()
number_of_vip_preferred_customers = country_vip_preferred_counts.max()

In [30]:
print(f"El país con la mayoría de Clientes VIP y Preferred combinados es {country_with_most_vip_preferred} con {number_of_vip_preferred_customers} clientes.")

El país con la mayoría de Clientes VIP y Preferred combinados es United Kingdom con 3920 clientes.
